# 強化学習におけるリプシッツ連続性

参考：
* [Lipschitz Continuity in Model-based Reinforcement Learning](https://arxiv.org/abs/1804.07193)

今回は状態空間が連続である場合の話について学びます．
状態空間が離散の場合は価値反復法が使えますが，連続の場合には使えません．

しかし，MDPの中にリプシッツ連続性のようなものが導入されてる場合は解くことができます．見てみましょう．

## リプシッツ連続性

数学的な準備をします．

---

**リプシッツ連続性**

２つの距離空間$\left(M_1, d_1\right)$と$\left(M_2, d_2\right)$を考えます．
関数$f: M_1 \mapsto M_2$は次で定義されるリプシッツ定数が有限である時にリプシッツ連続である，といいます．

$$
K_{d_1, d_2}(f):=\sup _{s_1 \in M_1, s_2 \in M_1} \frac{d_2\left(f\left(s_1\right), f\left(s_2\right)\right)}{d_1\left(s_1, s_2\right)}
$$

これは次と等価です：

$$
\forall s_1, \forall s_2 \quad d_2\left(f\left(s_1\right), f\left(s_2\right)\right) \leq K_{d_1, d_2}(f) d_1\left(s_1, s_2\right)
$$

---

**一様リプシッツ連続**

関数$f: M_1 \times \mathcal{A} \mapsto M_2$は次の定数が有限であるときに$\mathcal{A}$について一様リプシッツ連続であるといいます．

$$
K_{d_1, d_2}^{\mathcal{A}}(f):=\sup _{a \in \mathcal{A}} \sup _{s_1, s_2} \frac{d_2\left(f\left(s_1, a\right), f\left(s_2, a\right)\right)}{d_1\left(s_1, s_2\right)}
$$

---

